In [25]:
import pandas as pd
import numpy as np

In [26]:
how_df = pd.read_csv('AI_how.csv')
how_df.shape

(3623, 10)

In [27]:
impact_df = pd.read_csv('AI_impact.csv')
impact_df.shape

(3389, 10)

In [28]:
subject_df = pd.read_csv('AI_subject.csv')
subject_df.shape

(1674, 10)

In [38]:
# Set the maximum width for display
pd.set_option('display.max_colwidth', None)

In [ ]:
pd.reset_option('display.max_colwidth')

In [42]:
how_df.head(20)

,Video ID,Comment id,Comment creation time,Comment number of likes,Comment content,Replies,Video title,Video channel,Video creation time,Video number of views
0,JZ_flEGANBw,UgwuCX475lju2-tlWHV4AaABAg,2023-09-01T14:58:22Z,528,"A bunch of my professors have been freaking out about ChatGPT and asking students what they should do to adapt to technological changes in academic fields brought on by AI… this was my first week and AI policies were on the syllabus- academics are seriously freaking out about this. Needless to say, I emailed this video to all of my profs, I hope they watch it and incorporate your ideas into the classroom rather than trying and failing to ban the use of AI, or lazily allowing students to get out of work with it","['Would love to talk to your professors about their struggles and see how we can help. Let them know to reach out to us if they have questions', 'This new advancement of AI is going to RUIN education forever', 'Q', '❤❤', 'How did it go? How did they react?']",How to learn anything fast using ChatGPT | Full guide to studying with AI,Cajun Koi Academy,2023-09-01T14:31:41Z,346606
1,JZ_flEGANBw,UgzsV3g9-93xUX2m1QF4AaABAg,2023-09-03T22:40:25Z,9,AMAZING vid guys. This is definitely one of those vids that are like a black swan event since the AI is basically a more sophisticated search engine tailored specifically to the individual asking questions.\n\nPhenomenal!,NaN,How to learn anything fast using ChatGPT | Full guide to studying with AI,Cajun Koi Academy,2023-09-01T14:31:41Z,346606
2,JZ_flEGANBw,Ugyp7mBmxwvZIT9nL-94AaABAg,2023-09-09T03:35:40Z,5,You and Mike are EXTRAORDINARY as ever. Keep making videos for us. Love you both!,NaN,How to learn anything fast using ChatGPT | Full guide to studying with AI,Cajun Koi Academy,2023-09-01T14:31:41Z,346606
3,JZ_flEGANBw,UgzClGkl0S_ao7Rd8Ah4AaABAg,2023-09-02T11:30:41Z,25,Plot Twist: All research and the script was made by Chat GPT,NaN,How to learn anything fast using ChatGPT | Full guide to studying with AI,Cajun Koi Academy,2023-09-01T14:31:41Z,346606
4,JZ_flEGANBw,UgyxMaVPNbAHt61X0nt4AaABAg,2023-10-19T15:02:52Z,9,"I'm currently self-learning data science and this video has helped me find out about the uses of AI to improve the rate at which I learn concepts considerably. You've just earned a new subscriber, thank you.",NaN,How to learn anything fast using ChatGPT | Full guide to studying with AI,Cajun Koi Academy,2023-09-01T14:31:41Z,346606
5,JZ_flEGANBw,UgxzY2xz-bplUmIq0Lt4AaABAg,2023-09-03T12:30:23Z,8,"AI improved so many areas of my personal and work life. Being a graduate many years ago, you normally don't study as much anymore, with gpt I don't just have access to new concepts easily, I can also review books that I read long ago, practice new skills and test ideas. That's amazing! It's really great to see smart young people to be so methodical and understanding about it. 👍",NaN,How to learn anything fast using ChatGPT | Full guide to studying with AI,Cajun Koi Academy,2023-09-01T14:31:41Z,346606
6,JZ_flEGANBw,UgyCcUcxMWgFCx-6p5V4AaABAg,2023-09-02T15:29:40Z,9,I was asked to share my experience as a senior in college with incoming freshman for my degree and offer any tips for being a successful student. I told them all about your channel! This has seriously upped my study game!,NaN,How to learn anything fast using ChatGPT | Full guide to studying with AI,Cajun Koi Academy,2023-09-01T14:31:41Z,346606
7,JZ_flEGANBw,UgyWQ22KagnUFDJJ7fp4AaABAg,2023-09-02T04:29:02Z,8,One of the best videos on ' implementing AI in studying ' .\nCan't wait to use these tricks irl.\nThanks a bunch !!❤,NaN,How to learn anything fast using ChatGPT | Full guide to studying with AI,Cajun Koi Academy,2023-09-01T14:31:41Z,346606
8,JZ_flEGANBw,UgwGOOYEmeNNPlG9Bjt4AaABAg,2023-09-01T18:20:56Z,10,"Another banger video from you guys! By far the BEST video I've seen on using ChatGPT in effective ways for learning and stuyding aside from just using it as an advanced search engine. As a med stu